In [ ]:
import requests
import pandas as pd
import datetime
import json
import numpy as np
import hvplot.pandas
import panel as pn
import holoviews as hv

pn.extension(sizing_mode = 'stretch_width')

In [ ]:
def get_market_chart(coid_id, days):
        
    vs_currency='usd'
    
    if days == '1d' :
        num_of_days=1
    elif days == '7d':
        num_of_days=7
    elif days == '30d':    
        num_of_days=30
    elif days == '90d':    
        num_of_days=90 
    elif days == '1y':    
        num_of_days=365 
    else :   
        num_of_days='max'
        
    url = f"https://api.coingecko.com/api/v3/coins/{coid_id}/market_chart"
    payload={'vs_currency' : vs_currency, 'days' : num_of_days}
    
    response = requests.get(url, params=payload)
    data = response.json()
    
    timestamp_list = []
    price_list = []
    
    for row in data['prices'] :
        timestamp_list.append(datetime.datetime.fromtimestamp(row[0]/1000))
        price_list.append(row[1])
        
    raw_data = {
        'timestamp': timestamp_list,
        'price' : price_list
    }  
    
    #print(raw_data)
    df = pd.DataFrame(raw_data, columns=['timestamp','price']) 
    df = df.set_index('timestamp')
    
    title=f"{days}"
    return df, title

In [ ]:
def get_coin_price(coid_id) :
    
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={coid_id}&vs_currencies=usd&include_market_cap=true&include_24hr_vol=true&include_24hr_change=true&include_last_updated_at=true"
  
    response = requests.get(url)
    data = response.json()
    
    coin_price_df = pd.DataFrame(data)
    coin_price_df = coin_price_df.T
    
    return coin_price_df


In [ ]:
def get_coin_info(coid_id) :
    
    url = f"https://api.coingecko.com/api/v3/coins/{coid_id}?localization=false&tickers=false&market_data=false&community_data=false&developer_data=false&sparkline=false"
    response = requests.get(url)
    data = response.json()
    
    coinf_info_df = pd.DataFrame([data], columns=['id', 'symbol', 'name', 'asset_platform_id', 'platforms',
       'detail_platforms', 'block_time_in_minutes', 'hashing_algorithm',
       'categories', 'public_notice', 'additional_notices', 'description',
       'links', 'image', 'country_origin', 'genesis_date',
       'sentiment_votes_up_percentage', 'sentiment_votes_down_percentage',
       'market_cap_rank', 'coingecko_rank', 'coingecko_score',
       'developer_score', 'community_score', 'liquidity_score',
       'public_interest_score', 'public_interest_stats', 'status_updates',
       'last_updated'])
    
    coinf_info_df = coinf_info_df.set_index('id')
    
    coinf_info_df = coinf_info_df.drop('asset_platform_id', axis=1)
    coinf_info_df = coinf_info_df.drop('platforms', axis=1)
    coinf_info_df = coinf_info_df.drop('detail_platforms', axis=1)
    coinf_info_df = coinf_info_df.drop('public_notice', axis=1)
    coinf_info_df = coinf_info_df.drop('additional_notices', axis=1)

    return coinf_info_df


In [ ]:
bootstrap = pn.template.BootstrapTemplate(title='Crypto Info')

xs = np.linspace(0, np.pi)
coins_list = ['bitcoin', 'ethereum']
days_list = ['1d', '7d', '30d', '90d', '1y', 'All']

coin_id = pn.widgets.Select(name='Coins', value='bitcoin', options=coins_list)
days = pn.widgets.Select(name='Time Frame', value='1d', options=days_list)


@pn.depends(coin_id=coin_id, days=days)
def plot(coin_id, days):  
    coin_id_str=str(coin_id) 
    coin_df, title=get_market_chart(coid_id=coin_id_str, days=days)

    return coin_df.hvplot(title=title)

@pn.depends(coin_id=coin_id, days=days)
def coin_info(coin_id, days):  
    coin_id_str=str(coin_id)
    coin_price_df=get_coin_price(coin_id_str)
    return coin_price_df.head()


bootstrap.sidebar.append(coin_id)
bootstrap.sidebar.append(days)


bootstrap.main.append(
    
    pn.Row(
        pn.Card(plot, title='Coin Graph')      
    )
)
bootstrap.main.append(
    pn.Row(
        pn.Card(coin_info, title='coin_info')
        
    )
)
bootstrap.servable();